In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/mygithub/pytorchForAudioPractice

/content/drive/.shortcut-targets-by-id/1j0tKfNJ74iEAtyLmY4PY6L2fktF9jL1I/mygithub/pytorchForAudioPractice


In [3]:
!pip install torch torchaudio torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.autograd.profiler as profiler

In [5]:
def download_mnist_datasets():
  train_data = datasets.MNIST(train=True, download=True, transform=ToTensor(), root="data")
  validate_data = datasets.MNIST(train=False, download=True, transform=ToTensor(), root="data")
  return train_data, validate_data


In [6]:
from torch.nn.modules.linear import Linear
class FeedForwardNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.denseLayers = nn.Sequential(
        nn.Linear(28*28,256), # image size 28x28
        nn.ReLU(),
        nn.Linear(256,10) # 10 classes 0 to 9 
    )
    self.activation = nn.Softmax(dim=1)

  def forward(self, inputData):

    flattenedData = self.flatten(inputData)
    logits = self.denseLayers(flattenedData)
    predictions = self.activation(logits)

    return predictions

In [5]:
def trainOneEpoch(model, dataLoader, lossFn, optimiser, device):


  for input, target in dataLoader:

    input, target = input.to(device), target.to(device)

    #calculate loss
    predication = model(input)

    loss = lossFn(predication, target)

    # backpropagation and weight update

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  print(f"loss: {loss.item()}")

In [6]:
def train(model, dataLoader, lossFn, optimiser, device, epochs):
  for i in range(epochs):

    print(f"Epoch {i+1}")
    trainOneEpoch(model, dataLoader, lossFn, optimiser, device)
    print("---------------------------")
  
  print("Finished training")

# **Model Train**

In [ ]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 10

# Download data
train_data, _ = download_mnist_datasets()

# data load
dataLoader = DataLoader(train_data, batch_size=BATCH_SIZE)

# Build Model

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device}")

model = FeedForwardNN().to(device)

# Train

lossFn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


train(model, dataLoader, lossFn, optimiser, device, EPOCHS)

# Save Model

torch.save(model.state_dict(),"modelNN.pth")
print("Trained feed forward net saved at modelNN.pth")



Using cpu
Epoch 1
loss: 1.5117287635803223
---------------------------
Epoch 2
loss: 1.49992835521698
---------------------------
Epoch 3
loss: 1.4886845350265503
---------------------------
Epoch 4
loss: 1.4811148643493652
---------------------------
Epoch 5
loss: 1.475595474243164
---------------------------
Epoch 6
loss: 1.473410964012146
---------------------------
Epoch 7
loss: 1.4738837480545044
---------------------------
Epoch 8
loss: 1.4731812477111816
---------------------------
Epoch 9
loss: 1.4724453687667847
---------------------------
Epoch 10
loss: 1.4728208780288696
---------------------------
Finished training
Trained feed forward net saved at modelNN.pth


# Model Infere

In [ ]:
CLASS_MAPPING = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]

In [ ]:
def predict(model, input, target, classMapping):
  # switch eval 
  model.eval()

  # run infere without gradient evaluation
  with torch.no_grad():
    predictions = model(input)

    # Tensor (1, 10) --> [[0.1, 0.001, .... , 0.8]]
    predictedIndex = predictions[0].argmax(0)

    predicted = classMapping[predictedIndex]
    expected = classMapping[target]

  return  predicted, expected

In [ ]:
# Load Validate Data
_, validatingData = download_mnist_datasets()

# Load Model

feedForwardNetObj = FeedForwardNN()

dictStatLoaded = torch.load("modelNN.pth")

feedForwardNetObj.load_state_dict(dictStatLoaded)

# get sample from the validating data
input, target = validatingData[0][0], validatingData[0][1]

# Make inference

predicted, expected = predict(feedForwardNetObj, input, target, CLASS_MAPPING)

print(f"Predicted: '{predicted}', expected: '{expected}'")

Predicted: '7', expected: '7'


# Customized Urban Dataset

https://urbansounddataset.weebly.com/urbansound8k.html

# To download dataset

```
cd ---> path you need
!pip install opendatasets --upgrade --quiet

import opendatasets as od

dataset_url = 'https://goo.gl/8hY5ER'
od.download(dataset_url)
```

ref: https://jovian.ai/charmzshab/urban-sound-dataset

and also: https://www.kaggle.com/datasets/chrisfilo/urbansound8k

# Use Mel Spectrogram as transformer

In [7]:
from pandas.core.array_algos import transforms
import os
from torch.utils.data import Dataset
import pandas as pd
import torchaudio


class UrbanSoundDataset(Dataset):

  def __init__(self, annotationsFile,
               audioDir, transformer,
               sampleRate, sampleLength,
               device):
    self.annotations = pd.read_csv(annotationsFile)
    self.audioDir = audioDir
    self.device = device
    self.transformer = transformer.to(self.device)
    self.targetSampleRate = sampleRate
    self.sampleLength = sampleLength


  def __len__(self):
    return len(self.annotations)
    
  def __getitem__(self,index):
    audioSamplePath = self._getAudioSamplePath(index)
    label = self._getAudioSampleLabel(index)
    signal, sr = torchaudio.load(audioSamplePath)
    signal = signal.to(self.device)
    # need to resample if necessary
    signal = self._resampleIfNecessary(signal, sr)
    # need to mix the channels in case of stero or multi channel
    signal = self._mixChannelIfNecessary(signal)
    # need to check the size of the sample so that right padding added or trunked
    signal = self._adjustSampleLength(signal)
    # need to transform signal to the transforemer
    signal = self.transformer(signal)
    return signal, label

  def _getAudioSamplePath(self, index):
    fold = f"fold{self.annotations.iloc[index,5]}" # 5 where fold is located he cloumb # 5
    path = os.path.join(self.audioDir, fold, self.annotations.iloc[index, 0]) # where 0 is the raw of the .wav files names
    return path

  def _getAudioSampleLabel(self, index):
    return self.annotations.iloc[index, 6] # where the raw #6 is the classId

  def _resampleIfNecessary(self, signal, sr):
    if sr != self.targetSampleRate:
      resampler = torchaudio.transforms.Resample(sr, self.targetSampleRate).to(self.device)     
      signal = resampler(signal)
    return signal

  def _mixChannelIfNecessary(self, signal):
    # example, if signal is stero so its shape will be (2,16000)
    if signal.shape[0] > 1:
      signal = torch.mean(signal, dim=0, keepdim=True)
    return signal

  def _adjustSampleLength(self, signal):
    signalLen = signal.shape[1]
    # check in case of small sample so needed padding to be added
    if signalLen < self.sampleLength:
      numMissingSample = self.sampleLength - signalLen
      lastDimPadding = (0, numMissingSample)
      signal = torch.nn.functional.pad(signal, lastDimPadding)
    
    # in case of signal len greater than required len
    elif signalLen > self.sampleLength:
      signal = signal[:, :self.sampleLength]

    return signal


In [14]:
pwd

'/content/drive/.shortcut-targets-by-id/1j0tKfNJ74iEAtyLmY4PY6L2fktF9jL1I/mygithub/pytorchForAudioPractice'

In [19]:
from torch.cuda import is_available
ANNOTATION_FILE = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/audio"
SAMPLE_RATE = 22050
NUM_SAMPLE = 22050

melSpectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64)

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"Using device {device}")
usd = UrbanSoundDataset(ANNOTATION_FILE,
                        AUDIO_DIR,
                        melSpectrogram,
                        SAMPLE_RATE,
                        NUM_SAMPLE,
                        device)

print(f"There are {len(usd)} samples in the dataset.")
signal, label = usd[0]



Using device cuda
There are 8732 samples in the dataset.
 device = cuda


In [20]:
label

3

In [21]:
signal.shape

torch.Size([1, 64, 44])

# CNN For Audio Classification

In [8]:
from torch import nn
from torchsummary import summary

class CNNNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    # 4 conv blocks / flatten / linear / softmax
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv4 = nn.Sequential(
        nn.Conv2d(
            in_channels=64,
            out_channels=128,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(128*5*4, 10) # 128*5*4 is the out of the flatten, 10 is the classes of urban sound
    self.softmax = nn.Softmax(dim=1)
  
  def forward(self, inputData):
    x = self.conv1(inputData)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.flatten(x)
    logits = self.linear(x)
    predictions = self.softmax(logits)
    return predictions



In [28]:
cnn = CNNNetwork()
summary(cnn.to(device), (1,64,44))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 66, 46]             160
              ReLU-2           [-1, 16, 66, 46]               0
         MaxPool2d-3           [-1, 16, 33, 23]               0
            Conv2d-4           [-1, 32, 35, 25]           4,640
              ReLU-5           [-1, 32, 35, 25]               0
         MaxPool2d-6           [-1, 32, 17, 12]               0
            Conv2d-7           [-1, 64, 19, 14]          18,496
              ReLU-8           [-1, 64, 19, 14]               0
         MaxPool2d-9             [-1, 64, 9, 7]               0
           Conv2d-10           [-1, 128, 11, 9]          73,856
             ReLU-11           [-1, 128, 11, 9]               0
        MaxPool2d-12            [-1, 128, 5, 4]               0
          Flatten-13                 [-1, 2560]               0
           Linear-14                   

# Train CNN


In [9]:
from torch.cuda import is_available

ANNOTATION_FILE = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/metadata/UrbanSound8K.csv"
AUDIO_DIR = "/content/drive/MyDrive/mygithub/pytorchForAudioPractice/data/UrbanSound8K/audio"

SAMPLE_RATE = 22050
NUM_SAMPLE = 22050

BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device}")

# instantiating our dataset object and create data loader
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64
)

usd = UrbanSoundDataset(ANNOTATION_FILE,
                        AUDIO_DIR,
                        mel_spectrogram,
                        SAMPLE_RATE,
                        NUM_SAMPLE,
                        device)

train_dataloader = DataLoader(usd, BATCH_SIZE)

# construct model and assign it to device
cnn = CNNNetwork().to(device)
print(cnn)

# initialise loss funtion + optimiser
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(cnn.parameters(),
                              lr=LEARNING_RATE)

# train model
train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

# save model
torch.save(cnn.state_dict(), "cnn.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Using cpu
CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)
Epoch 1
loss: 2.4313237667083